# Refactoring CSV For Historical Data

In [2]:
import os 
import pandas as pd 
import numpy as np 
import shutil 
from pathlib import Path

In [3]:
print(f"Current Working Directory is: {os.getcwd()}")

Current Working Directory is: /Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/code


In [6]:
DATA_ROOT = Path("/Users/saikeerthan/Coding/NYP/IOTA/IoT_Weather_project/model-training/datasets/official_data.csv")

if DATA_ROOT.exists():
    print("Data Root Exists!")
else: 
    print("Data Root Doesn't Exist!")

Data Root Exists!


In [7]:
df = pd.read_csv(DATA_ROOT)

df

,time,cidx,cattr,temp,humi,pres,windspeed,winddirection,rainfall,uvindex
0,2026-01-24 20:23:02,1,7,28.6,74,1016.746582,5.5,54,NaN,NaN
1,2026-01-24 20:28:02,2,7,28.5,74,1016.802490,5.3,53,NaN,NaN
2,2026-01-24 20:33:02,3,7,28.6,74,1016.859131,5.0,53,NaN,NaN
3,2026-01-24 20:38:02,4,7,28.5,74,1016.934326,4.9,50,NaN,NaN
4,2026-01-24 20:43:02,5,7,28.6,74,1016.971436,5.4,50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2271,2026-02-02 10:26:15,1045,7,38.5,47,1018.157959,6.9,75,NaN,2.0
2272,2026-02-02 10:31:15,1046,7,38.5,47,1018.206055,8.2,73,NaN,2.0
2273,2026-02-02 10:36:15,1047,7,38.5,47,1018.149414,8.2,66,NaN,2.0
2274,2026-02-02 10:41:15,1048,7,38.5,47,1018.127930,8.1,47,NaN,2.0


In [8]:
#checking for duplicates 

duplicates = df.duplicated().sum()

print(f"Duplicates present in DF: {duplicates}")

Duplicates present in DF: 0


In [9]:
# checking for missing values 

missing = df.isnull().sum().sum()

print(f"Missing values in df: {missing}")

# missing values is because of last two columns

Missing values in df: 3768


In [10]:
#columns in df 

columns = df.columns

print("Columns in Dataset:\n")
for i, col in enumerate(df.columns, 1):
    print(f"{i}. {col}")

Columns in Dataset:

1. time
2. cidx
3. cattr
4. temp
5. humi
6. pres
7. windspeed
8. winddirection
9. rainfall
10. uvindex


In [11]:
# drop entries where cattr is below 7

df = df[df["cattr"] >=7]

df

,time,cidx,cattr,temp,humi,pres,windspeed,winddirection,rainfall,uvindex
0,2026-01-24 20:23:02,1,7,28.6,74,1016.746582,5.5,54,NaN,NaN
1,2026-01-24 20:28:02,2,7,28.5,74,1016.802490,5.3,53,NaN,NaN
2,2026-01-24 20:33:02,3,7,28.6,74,1016.859131,5.0,53,NaN,NaN
3,2026-01-24 20:38:02,4,7,28.5,74,1016.934326,4.9,50,NaN,NaN
4,2026-01-24 20:43:02,5,7,28.6,74,1016.971436,5.4,50,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2271,2026-02-02 10:26:15,1045,7,38.5,47,1018.157959,6.9,75,NaN,2.0
2272,2026-02-02 10:31:15,1046,7,38.5,47,1018.206055,8.2,73,NaN,2.0
2273,2026-02-02 10:36:15,1047,7,38.5,47,1018.149414,8.2,66,NaN,2.0
2274,2026-02-02 10:41:15,1048,7,38.5,47,1018.127930,8.1,47,NaN,2.0


In [12]:
columns_to_delete = ["cattr", "windspeed", "winddirection", "rainfall", "uvindex", "cidx"]

df = df.drop(columns=columns_to_delete)

columns = df.columns
print(f"Remaining Columns:{columns}")

Remaining Columns:Index(['time', 'temp', 'humi', 'pres'], dtype='object')


In [13]:
# check the dtype of every column in the df 

print(df.dtypes)

time     object
temp    float64
humi      int64
pres    float64
dtype: object


In [14]:
df["time"] = pd.to_datetime(df["time"])

print(f"New Dtypes for columns in df: {df.dtypes}")

New Dtypes for columns in df: time    datetime64[ns]
temp           float64
humi             int64
pres           float64
dtype: object


In [15]:
# 2) Floats -> float32 (half the memory of float64)
df["temp"] = pd.to_numeric(df["temp"], errors="coerce").astype("float32")
df["pres"] = pd.to_numeric(df["pres"], errors="coerce").astype("float32")

# 3) Humidity -> nullable UInt8 (compact, supports NaN)
df["humi"] = pd.to_numeric(df["humi"], errors="coerce").round().astype("UInt8")

In [16]:
print(df.dtypes)

time    datetime64[ns]
temp           float32
humi             UInt8
pres           float32
dtype: object


In [17]:
assert df["humi"].dropna().between(0, 100).all(), "Humidity out of expected range"

In [18]:
df

,time,temp,humi,pres
0,2026-01-24 20:23:02,28.6,74,1016.746582
1,2026-01-24 20:28:02,28.5,74,1016.802490
2,2026-01-24 20:33:02,28.6,74,1016.859131
3,2026-01-24 20:38:02,28.5,74,1016.934326
4,2026-01-24 20:43:02,28.6,74,1016.971436
...,...,...,...,...
2271,2026-02-02 10:26:15,38.5,47,1018.157959
2272,2026-02-02 10:31:15,38.5,47,1018.206055
2273,2026-02-02 10:36:15,38.5,47,1018.149414
2274,2026-02-02 10:41:15,38.5,47,1018.127930


In [19]:
df = df.sort_values("time").reset_index(drop=True)

df

,time,temp,humi,pres
0,2026-01-24 20:23:02,28.6,74,1016.746582
1,2026-01-24 20:28:02,28.5,74,1016.802490
2,2026-01-24 20:33:02,28.6,74,1016.859131
3,2026-01-24 20:38:02,28.5,74,1016.934326
4,2026-01-24 20:43:02,28.6,74,1016.971436
...,...,...,...,...
2192,2026-02-02 10:26:15,38.5,47,1018.157959
2193,2026-02-02 10:31:15,38.5,47,1018.206055
2194,2026-02-02 10:36:15,38.5,47,1018.149414
2195,2026-02-02 10:41:15,38.5,47,1018.127930


In [20]:
df["pres"] = df["pres"].round(3)


df

,time,temp,humi,pres
0,2026-01-24 20:23:02,28.6,74,1016.747009
1,2026-01-24 20:28:02,28.5,74,1016.802002
2,2026-01-24 20:33:02,28.6,74,1016.859009
3,2026-01-24 20:38:02,28.5,74,1016.934021
4,2026-01-24 20:43:02,28.6,74,1016.971008
...,...,...,...,...
2192,2026-02-02 10:26:15,38.5,47,1018.158020
2193,2026-02-02 10:31:15,38.5,47,1018.205994
2194,2026-02-02 10:36:15,38.5,47,1018.148987
2195,2026-02-02 10:41:15,38.5,47,1018.127991


In [22]:
df.to_csv("official_historical_ui.csv")